# Land use and Land Cover Classification

The availability of free satellite data has increased its use in several applications in the domains of agriculture, disaster recovery, climate change, urban development, or environmental monitoring can be realized. However, to fully utilize the data for the previously mentioned domains, first satellite images must be processed and transformed into structured semantics. One type of such fundamental semantics is Land Use and Land Cover Classification. The aim of land use and land cover classification is to automatically provide labels describing the represented physical land type or how a land area is used (e.g., residential, industrial)

A satellite image dataset for the task of land use and land cover classification was presented in [[1]](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8519248). The EuroSAT dataset is based on Sentinel-2 satellite images and consists of 27,000 labeled images with a total of 10 different classes listed below where the patches are 64x64 pixels each.

![alt text](./Images/dataset.png "The EuroSAT Dataset")

In this assignment you are going to use the optical bands of Sentinel-2 which are computed by combining the bands red (B04), green (B03) and blue (B02) from the Sentinel-2 product. More information about the Sentinel-2 bands can be found [here](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/resolutions/spatial). You will then train different Convolutional Neural Network (CNN) models to classify every 64x64 patches in one of the following classes:

1. AnnualCrop
2. Forest
3. Herbaceous Vegetation
4. Highway
5. Industrial
6. Pasture
7. Permanent Crop
8. Residential
9. River
10. SeaLake



## Instructions
The EuroSAT dataset is based on Sentinel-2 satellite images and consists of 27,000 labeled images with a total of 10 different classes. The dataset is structured as follows:
1. `train.txt`: this file contains a list of images that will be used to train the Convolutional Neural Network (CNN) models.
2. `test.txt`: this file contains a list of images that will be used to test the Convolutional Neural Network (CNN) models.
3. A list of 10 folders, each one containing the images pertaining to each class.

All code needs to be developed in Python 3 and run on a Ubuntu 20.04 environment or later versions of Ubuntu. The student is requested send the jupyter notebook using the template provided. Any textual or visual information and equations that the student might need to convey is expected to be written using the markdown language within the same Juputer Notebook. The Juputer Notebook should be named as follows

`name-surname.ipynb`

The list of packages that are allowed for this assignment are: `matplotlib`, `os`, `numpy`, `torch`, `open-cv`, `torchvision` and any other packages agreed with the lecturer.


## Assignment

**Q1:** A lot of effort in solving any machine learning and computer vision problem goes into preparing the data. PyTorch provides a simple mechanism to define a custom dataset using `torch.utils.data.Dataset`, which is an abstract class representing a dataset. Your custom dataset should inherit `Dataset` and override the following methods:

- `__init__` so that it initializes the dataset
- `__len__` so that len(dataset) returns the size of the dataset.
- `__getitem__` to support the indexing such that dataset[i] can be used to get i-th sample.


Write a class `DataLoaderClassification` that can be used to
- load the list of image filenames and the corresponding lables in two lists in `__init__`
- load a batch of images and corresponding lables when one calls `__getitem__`
- returns the length of the dataset using `__len__`

Write the code in one or more cells.

In [23]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import random

Debug = False

In [37]:
###############################################
#  DataLoaderClassification() ::              #
###############################################
class DataLoaderClassification(Dataset):
    def __init__(self, data_dir, file_list_path):
        # Args:
        #    data_dir :: Path to the directory containing image subfolders.
        #    file_list_path :: Path to the text file containing image paths (e.g., train.txt or test.txt).

        self.data_dir = data_dir        
        self.image_paths = []
        self.labels = []
        
        # A set to collect unique labels
        label_set = set()
        
        with open(file_list_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line:
                    updated_path = line.replace('EuroSAT/', '')  
                    self.image_paths.append(updated_path)
                    
                    label = os.path.split(updated_path)[0].replace('./Data/', '')  
                    self.labels.append(label)
                    label_set.add(label)  # Collect unique labels
                    
        # Create a mapping from label strings to integers
        self.label_to_index = {label: idx for idx, label in enumerate(sorted(label_set))}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_full_path = self.image_paths[idx]
        label = self.labels[idx]
        
        image = cv2.imread(img_full_path)
        if image is None:
            raise FileNotFoundError(f"Image at path {img_full_path} could not be loaded.")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)# Convert to RGB

        # image to tensor
        transform = transforms.ToTensor()
        image = transform(image)
        
        # Convert label from string to integer using the label mapping and, then, to tensor
        label = self.label_to_index[label]
        label = torch.tensor(label)
        
        return image, label

In [38]:
data_dir = 'Data'
train_file_list = 'Data/train.txt'
test_file_list = 'Data/test.txt'

train_dataset = DataLoaderClassification(data_dir=data_dir, file_list_path=train_file_list)
test_dataset  = DataLoaderClassification(data_dir=data_dir, file_list_path=test_file_list)

print(f"Number of items in train_dataset: {len(train_dataset)}")
print(f"Number of items in test_dataset: {len(test_dataset)}")


# test for getitem
if Debug:
    test_index = random.randint(0, 21599)
    print(f"Check item {test_index} from train_dataset")
    image, label = train_dataset[test_index]
    print(f"Image {test_index} -> Shape: {image.shape}, Label: {label}")

Number of items in train_dataset: 21600
Number of items in test_dataset: 5400


**Q2:** Write the code in one cell that uses the list of files included in `train.txt` and `test.txt` to create a Pytorch dataloader for the training and testing data, respectively.

**Loading in batches**

DataLoader: Shuffling while training to prevent learning parterns related to the order.
When testing, shuffling is typically set to False to maintain the order of the test samples.

In [39]:
#num_workers = 0   # number of subprocesses to use for data loading
batch_size = 20   # how many samples per batch to load

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# test: Iterate through the dataset in batches
if Debug:
    for batch_idx, (images, labels) in enumerate(train_loader):
        print(f"\n Batch {batch_idx + 1}  \n   Image batch shape: {images.shape}, \n   Labels: {labels}")
        if batch_idx == 2:
            break


**Q3:** PyTorch provides the elegantly designed modules and classes, including `torch.nn`, to help you create and train neural networks. An `nn.Module` contains layers, and a method `forward(input)` that returns the output. Write the `CNN` class to define a Convolutional Neural Network (CNN) where the first convolutional layer (`conv1`) takes 3 input channels, outputs 16 output channels and has a kernel size of 5. The output of `conv1` is fed into a ReLU followed by a Max-pooling operator. The second convolutional layer in this network (`conv2`) should have 32 filters with a kernel size of 5 followed by a ReLU and a max-pooling operator. The last layer is a fully-connected layer (`fc1`) with 10 output neurons. In this code you should define the `__init__` and `forward` member functions.

More information about `torch.nn` can be found [here](https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html).

In [40]:
import torch.nn as nn
import torch.nn.functional as F


###############################################
#  SimpleCNN() :: Arquitecture defined in Q3 #
###############################################
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        self.fc1 = nn.Linear(in_features=32 * 13 * 13, out_features=10)

    def forward(self, x):
                                                     # --> 64x64x3 -->
        x = F.relu(self.conv1(x))                    # --> 16x60x60 (kernel_size=5 without padding)
        x = F.max_pool2d(x, kernel_size=2, stride=2) # --> 16x30x30 
        x = F.relu(self.conv2(x))                    # --> 32x26x26 (kernel_size=5 without padding)
        x = F.max_pool2d(x, kernel_size=2, stride=2) # --> 32x13x13
        x = x.view(-1, 32 * 13 * 13)                 # --> 1x5408
        x = self.fc1(x)                              # --> 1x10
        return x

Pooling with kernel size of 2 and stride of 2 (typicall choice) -> downsamples the feature map by a factor of 2

**Q4:** Write the code in one or more cells to train the CNN specified in **Q3**. Plot the accuracy against the number of epochs.  Save the best performing model in the folder `./Model/Simple-CNN/model.pth` and print the highest accuracy achieved after 100 epochs using the markdown language.

In [41]:
train_on_gpu = torch.cuda.is_available() # I have an M3, so no cuda for me 
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    
model = SimpleCNN()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()
else:
    model.to('cpu')

CUDA is not available.  Training on CPU ...
SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=5408, out_features=10, bias=True)
)


In [42]:
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim


# split train in validation + train
valid_size = 0.2  # percentage of training set to use as validation

num_train = len(train_dataset)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
valid_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=valid_sampler)

# Debug
if Debug:
    for batch_idx, (images, labels) in enumerate(train_loader):
        print(f"\n Train batch {batch_idx + 1}  \n   Image batch shape: {images.shape}, \n   Labels: {labels}")
        if batch_idx == 2:
            break
    for batch_idx, (images, labels) in enumerate(valid_loader):
        print(f"\n Validation batch {batch_idx + 1}  \n   Image batch shape: {images.shape}, \n   Labels: {labels}")
        if batch_idx == 2:
            break


# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

# number of epochs to train the model
n_epochs = 30
valid_loss_min = np.Inf


In [51]:

#######################################
#  train_model() :: Training function #
#######################################
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs, save_path="./Model/Simple-CNN/", save_name ="model.pth"):
    
    # Keep track of training and validation loss
    best_accuracy = 0.0
    valid_loss_min = np.Inf  # Initialize minimum validation loss
    train_losses, valid_losses = [], []          # Store loss values for plotting
    train_accuracies, valid_accuracies = [], []  # Store accuracy values for plotting

    for epoch in range(1, num_epochs+1):

        ###################
        # train the model # --> train_loader
        ###################
        model.train()  # Set the model to training mode
        model.train()  # Set the model to training mode
        train_loss = 0.0
        valid_loss = 0.0
        correct_train = 0
        total_train = 0––        
        
        # Training phase
        for images, labels in train_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                images, labels = images.cuda(), labels.cuda()

            optimizer.zero_grad()               # Clear the gradients
            output = model(images)              # Forward pass, calculate ouput
            loss = criterion(output, labels)    # Calculate the loss
            loss.backward()                     # Backpropagation
            optimizer.step()                    # Update weights

            train_loss += loss.item() * images.size(0)  # Track training loss
            
            _, predicted = torch.max(output, 1)  # Get predicted class
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        ######################
        # validate the model # --> valid_loader
        ######################
        model.eval()  # Set the model to evaluation mode
        correct_valid = 0
        total_valid = 0

        with torch.no_grad():  # Disable gradient calculation for validation
            for images, labels in valid_loader:
                if train_on_gpu:
                    images, labels = images.cuda(), labels.cuda()

                output = model(images)                      # Forward pass
                loss = criterion(output, labels)            # Calculate the validation loss
                valid_loss += loss.item() * images.size(0)  # Track validation loss

                _, predicted = torch.max(output, 1)         # Get predicted class
                correct_valid += (predicted == labels).sum().item()
                total_valid += labels.size(0)

        # Calculate average losses
        train_loss = train_loss / len(train_loader.sampler)
        valid_loss = valid_loss / len(valid_loader.sampler)

        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

        # Calculate training and validation accuracy
        train_accuracy = 100 * correct_train / total_train
        valid_accuracy = 100 * correct_valid / total_valid

        train_accuracies.append(train_accuracy)
        valid_accuracies.append(valid_accuracy)

        # Print training/validation statistics
        print(f'Epoch: {epoch} \tTraining Loss: {train_loss:.6f} \tValidation Loss: {valid_loss:.6f} \tTraining Accuracy: {train_accuracy:.2f}% \tValidation Accuracy: {valid_accuracy:.2f}%')

        # Save the model if validation loss has decreased
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        save_full_path = os.path.join(save_path, save_name)
        
        if valid_loss < valid_loss_min:
            print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model ...')
            torch.save(model.state_dict(), save_full_path)
            valid_loss_min = valid_loss  # Update minimum validation loss

    return train_losses, valid_losses, train_accuracies, valid_accuracies



    
# RUN TRAINING
train_losses, valid_losses, train_accuracies, valid_accuracies = train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=n_epochs)



Epoch: 1 	Training Loss: 0.799781 	Validation Loss: 0.810625 	Training Accuracy: 71.40% 	Validation Accuracy: 70.81%
Validation loss decreased (inf --> 0.810625). Saving model ...
Epoch: 2 	Training Loss: 0.732339 	Validation Loss: 0.793114 	Training Accuracy: 74.10% 	Validation Accuracy: 71.60%
Validation loss decreased (0.810625 --> 0.793114). Saving model ...
Epoch: 3 	Training Loss: 0.678053 	Validation Loss: 0.694072 	Training Accuracy: 75.89% 	Validation Accuracy: 75.25%
Validation loss decreased (0.793114 --> 0.694072). Saving model ...
Epoch: 4 	Training Loss: 0.636971 	Validation Loss: 0.739332 	Training Accuracy: 77.67% 	Validation Accuracy: 74.03%
Epoch: 5 	Training Loss: 0.595344 	Validation Loss: 0.652789 	Training Accuracy: 79.11% 	Validation Accuracy: 77.71%
Validation loss decreased (0.694072 --> 0.652789). Saving model ...
Epoch: 6 	Training Loss: 0.563354 	Validation Loss: 0.715615 	Training Accuracy: 80.46% 	Validation Accuracy: 74.65%
Epoch: 7 	Training Loss: 0.5301

In [52]:
# Plot accuracy vs epochs
def plot_accuracy(train_accuracies, valid_accuracies, num_epochs):
    epochs = range(1, num_epochs + 1)
    plt.plot(epochs, train_accuracies, 'r', label='Training Accuracy')
    plt.plot(epochs, valid_accuracies, 'b', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.show()
    
# Plot loss vs epochs
def plot_loss(train_losses, valid_losses, num_epochs):
    epochs = range(1, num_epochs + 1)
    plt.plot(epochs, train_losses, 'r', label='Training Loss')
    plt.plot(epochs, valid_losses, 'b', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Plot accuracy and loss
plot_accuracy(train_accuracies, valid_accuracies, num_epochs=n_epochs)
plot_loss(train_losses, valid_losses, num_epochs=n_epochs)

NameError: name 'plt' is not defined

**Q5:** Your role as a researcher is to improve the performance of the current neural network. Explain the architecture that provided the best performance and describe the modifications that you think provided the gain.
    

In [ ]:
# Code goes here